# Creating a stellar color-magnitude (or Hertzsprung-Russell) diagram

The [Hertzsprung-Russell diagram](https://en.wikipedia.org/wiki/Hertzsprung–Russell_diagram) is a fundamental diagram in astronomy that displays important relationships between the stellar color (or temperature) and absolute brightness (or luminosity). 

In this exercise, we will use existing stellar catalogs to produce the H-R diagram. 


In [ ]:
# As a hint, we include the code block for Python modules that you will likely need to import:   
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline  

# For downloading files
from astropy.utils.data import download_file
from astropy.io import fits

import pyvo as vo
from pyvo import registry

## There are a number of relatively unimportant warnings that 
## show up, so for now, suppress them:
import warnings
warnings.filterwarnings("ignore", module="astropy.io.votable.*")
warnings.filterwarnings("ignore", module="pyvo.utils.xml.*")

## Step 1: Find appropriate catalogs

We want to find a star catalog that has the available data to produce the H-R diagram, i.e., the absolute magnitudes (or both apparent magnitudes AND distances, so we can calculate the absolute magnitudes) in two optical bands (e.g., B and V). This would give us color. Or we need B- OR V- band magnitude and the stellar temperature. 

To simplify this problem, we want to find a catalog of an open cluster of stars, where all the stars were born around the same time and are located in one cluster. This simplifies the issue of getting accurate distances to the stars. One famous cluster is the Pleiades in the constellation of Taurus. So first we start by searching for an existing catalog with data on Pleiades that will provide the necessary information about the stars: magnitudes in two bands (e.g., B and V), which can be used to measure color, or temperature of the star plus one magnitude. 


### DATA DISCOVERY STEPS: 

Here is useful link for [how the pyvo registry search works](https://pyvo.readthedocs.io/en/latest/registry/index.html).

In [ ]:
# Write some code to perform a registry search using 
# appropriate keywords. Also apply the includeaux=True 
# option to return the maximum number of service: 


# Print the size of the output table: 


Note: The includeaux=True includes auxiliary services. 

This returns > 300 resources. However, we know that we want to eventually access the search using additional column information (i.e., beyond RA and Dec, which is all that Simple Cone Search returns). Therefore we need the service to be a TAP service, specifically. 

In [ ]:
# Similar to above, this time specify the servicetype
# to use the table access protocol: 

# Again, print the length of the output table: 

# Print the table: 


Note that this is now >100 services, but the data within the columns will be accessible for our purposes. 

#### Next, we need to find which of these has the columns of interest, i.e. magnitudes in two bands to create the color-magnitude diagram. 

For each resource element (i.e. row in the table above), there are useful attributes, which are [described here]( https://pyvo.readthedocs.io/en/latest/api/pyvo.registry.regtap.RegistryResource.html#pyvo.registry.regtap.RegistryResource):

Based on these, we can create a search based on keywords in the description. 

In [ ]:
# Write a function that searches the table descriptions
# for specific keywords.

In [ ]:
# Use the function above to output the table indices that 
# use the keywords "magnitude" and "color". 

# How many tables do you get?

So using this we can reduce the matched tables to ones that are a bit more catered to our experiment. Note, that there is redundancy in some resources since these are available via multiple services and/or publishers. Therefore a bit more cleaning can be done to provide only the unique matches. 

In [ ]:
# Print the 'ivoid' column for these tables 


In [ ]:
# Repeat the steps above but this time make sure that you 
# output only unique tables. 


In [ ]:
# Output the descriptions of the resulting matches:


<i> RESULT: Based on these, the first one (by Eichhorn et al) looks like a good start. </i>

### At this point, you can proceed to Step 2. 

-- OR --

### Try a different data discovery method! 

### Alternative Method: Use ADS to search for appropriate paper and access data via NED.

There are multiple paths for the data discovery. So it may also be that you know the paper that has the data you are interested in and want to access via the bibcode or authors, etc. 

In this case, let's assume that we have the information that the Eichhorn+1970 paper has the data that we need to create the H-R diagram: https://ui.adsabs.harvard.edu/abs/1970MmRAS..73..125E/abstract

We can either search by bibcode (1970MmRAS..73..125E) or "Eichhorn" to get the access_urls that will allow us to work with the data. 

Before this step, if may help to see the names of the fields available to use. Notice the following fields: 

"source_value" contains the bibcode information that we want; "creator_seq" lists the authors; 

and 

"access_url" provides the url from where the data can be accessed. 


In [ ]:
# Print the fieldnames for the tap services. 

First, Try using bibcode: 

In [ ]:
bibcode = '1970MmRAS..73..125E' # Eichhorn

#  
# print "short_name", "source_value" and "access_url" information 
# for the table entry that matches this bibcode. 
#
#  Note that using the to_table() lets you search the result 
#   easily using all columns.  But in the end, you want to get
#   back not an astropy table row, which you cannot use, but the
#   original RegistryResult that has the callable TAP service. 


Note that the URL is a generic TAP url for Vizier.  All of its tables can be accessed by that same TAP services.  It'll be in the ADQL query itself that you specify the table name.  We'll see this below.

Next, try using Author name: 

In [ ]:
author = 'Eichhorn'

#  
# print "short_name", "reference_url" and "access_url" information 
# for the table entry that matches this author name. 
#
# Hint: The pyVO Registry resource page may help:
# https://pyvo.readthedocs.io/en/latest/api/pyvo.registry.regtap.RegistryResource.html#pyvo.registry.regtap.RegistryResource.search


These examples provide a few ways to access the information of interest. 

Below are a few other ways to see what the tap_service table contains. 

1. To view the column information: tap_services.to_table().columns() shows the metadata contained in the tap service. We will reference some of this columns below as we try to find the appropriate table. 

2. tap_services[index].describe(): The table with the tap_services output has, in our case, 72 tables listed and each includes metadata containing some human readable description. You can get the description for one case or for all the records by iterating through the resource. In the former case, we show the description for the Eichhorn data. The latter case also follows.  
    

In [ ]:
# Print the full description for the Eichhorn+1970 example. 

## Step 2: Acquire the relevant data and make a plot!

In [ ]:
# Get the table name for the Eichhorn+1970 data
# and output the table information (i.e. data in columns) 


We can access the column data as array using the .getcolumn(colname) attribute, where the colname is given in the table above. In particular the "CI" is the color index and "Ptm" is the photovisual magnitude. See [here](https://vizier.u-strasbg.fr/viz-bin/VizieR?-source=I/90) for details about the columns. 

In [ ]:
# get color and magnitude column information for
# stars in this table. 

### Plotting... 
Note: The magnitudes here are apparent and therefore in plotting, the color-magnitude diagram is typically brightness increasing upwards (higher y-axis) so we will flip the y-axis here. 

In [ ]:
plt.ylim(15, 0)
plt.ylabel("V [apparent mag]")
plt.xlabel("B-V")
#
# plot color and magnitude data: 



## Step 3. Compare with other color-magnitude diagrams for Pleiades:

There is nice discussion here: http://www.southastrodel.com/Page03009a.htm  about the color-magnitude diagram. Their Fig 4 looks slightly cleaner because part of this investigation was to select the 270 stars that are vetted members and restricted to stellar types more massive than K0. 

The dataset is from Raboud+1998 (1998A&A...329..101R)

Therefore in this next step, we will use the bibcode to select this data and overplot with the previous data to compare. 

We note that we did not discover this table above because alas! the description does not include the keywords (e.g. color, magnitude) that we used to refine our search. However, this table does have color and magnitude columns. 

In [ ]:
bibcode = '1998A&A...329..101R' # Raboud

# Repeat steps above with this other dataset


In [ ]:

plt.ylim(15, 0)
plt.ylabel("V [apparent mag]")
plt.xlabel("B-V")

# Plot the Eichhorn data as black circles and 
# Raboud data as red squares. 


## BONUS: Step 4: The CMD as a distance indicator! 

Since the y-axis above is apparent magnitude, we can use the obvious features (e.g., main sequence curve) to translate the apparent magnitudes to absolute magnitudes (by comparing to published H-R diagrams given in absolute magnitudes) and measure the distance to Pleiades! 


In [ ]:

sun_color = 0.65  # from http://www.astro.ucla.edu/~wright/magcolor.htm

# Based on the color-magnitude diagram, what would be
# the Sun's apparent magnitude at the distance of the Pleiades?
# i.e. if you overplot the Sun on the CMD above
# what's the Sun's magnitude, corresponding to the sun's color
# given as B-V=0.65? 

# Overplot the sun in the plot above for reference. 



In [ ]:
# Measure the distance to the Pleaides, using the Sun
# as a reference. We know the Sun's absolute magnitude: 
Vabs = 4.8   ## Sun @ B-V = 0.65 (taken from Wikipedia)

# Using the Sun's apparent magnitude from your estimate above
# what's the distance to Pleiades in pc? 

True distance to Pleaides is 136.2 pc  (https://en.wikipedia.org/wiki/Pleiades ). Not bad!